In [1]:
!pip install sentence-transformers scikit-learn pandas

In [3]:
import pandas as pd

data = [
    ["Explain backpropagation", "Explanation", "Neural Networks", "Intermediate"],
    ["Give example of gradient descent", "Example", "Optimization", "Beginner"],
    ["I don't understand loss functions", "Doubt", "Neural Networks", "Beginner"],
    ["Revise activation functions", "Revision", "Neural Networks", "Beginner"],
    ["Explain overfitting", "Explanation", "Regularization", "Intermediate"],
    ["Example of dropout", "Example", "Regularization", "Intermediate"],
    ["I'm confused about learning rate", "Doubt", "Optimization", "Beginner"],
    ["Revise neural network basics", "Revision", "Neural Networks", "Beginner"]
]

df = pd.DataFrame(data, columns=["query", "intent", "topic", "difficulty"])
df

,query,intent,topic,difficulty
0,Explain backpropagation,Explanation,Neural Networks,Intermediate
1,Give example of gradient descent,Example,Optimization,Beginner
2,I don't understand loss functions,Doubt,Neural Networks,Beginner
3,Revise activation functions,Revision,Neural Networks,Beginner
4,Explain overfitting,Explanation,Regularization,Intermediate
5,Example of dropout,Example,Regularization,Intermediate
6,I'm confused about learning rate,Doubt,Optimization,Beginner
7,Revise neural network basics,Revision,Neural Networks,Beginner


In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

embeddings = model.encode(df["query"].tolist())

print(embeddings.shape)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(8, 384)


In [5]:
from sklearn.linear_model import LogisticRegression

# Create three classifiers
intent_clf = LogisticRegression()
topic_clf = LogisticRegression()
difficulty_clf = LogisticRegression()

# Train them
intent_clf.fit(embeddings, df["intent"])
topic_clf.fit(embeddings, df["topic"])
difficulty_clf.fit(embeddings, df["difficulty"])

print("Models trained successfully")

Models trained successfully


In [6]:
def analyze_query(query):
    emb = model.encode([query])

    intent = intent_clf.predict(emb)[0]
    topic = topic_clf.predict(emb)[0]
    difficulty = difficulty_clf.predict(emb)[0]

    return {
        "intent": intent,
        "topic": topic,
        "difficulty": difficulty
    }

# Test the system
result = analyze_query("I don't understand backpropagation")
print(result)

{'intent': 'Explanation', 'topic': 'Neural Networks', 'difficulty': 'Beginner'}


In [7]:
def recommend_next_step(score, attempts, time_spent):

    if score < 50:
        action = "Revision"
        difficulty = "Decrease"
    elif score < 75:
        action = "Practice"
        difficulty = "Same"
    else:
        action = "Advance"
        difficulty = "Increase"

    if action == "Revision":
        next_topic = "Previous fundamentals"
    elif action == "Practice":
        next_topic = "Similar difficulty topic"
    else:
        next_topic = "Next advanced topic"

    return {
        "next_topic": next_topic,
        "action": action,
        "difficulty_adjustment": difficulty
    }

# Test the function
print(recommend_next_step(score=40, attempts=3, time_spent=10))

{'next_topic': 'Previous fundamentals', 'action': 'Revision', 'difficulty_adjustment': 'Decrease'}


In [8]:
def full_system(query, score, attempts, time_spent):

    # Step 1: Analyze student question
    analysis = analyze_query(query)

    # Step 2: Recommend next step
    recommendation = recommend_next_step(score, attempts, time_spent)

    # Combine results
    return {
        "query_analysis": analysis,
        "recommendation": recommendation
    }

# Test the full system
output = full_system(
    query="I don't understand gradient descent",
    score=45,
    attempts=2,
    time_spent=12
)

print(output)

{'query_analysis': {'intent': 'Doubt', 'topic': 'Neural Networks', 'difficulty': 'Beginner'}, 'recommendation': {'next_topic': 'Previous fundamentals', 'action': 'Revision', 'difficulty_adjustment': 'Decrease'}}


In [10]:
# Interactive version of the system

student_query = input("Enter student question: ")
score = int(input("Enter quiz score (0-100): "))
attempts = int(input("Enter number of attempts: "))
time_spent = int(input("Enter time spent (minutes): "))

result = full_system(student_query, score, attempts, time_spent)

print("\nSystem Output:")
print(result)

Enter student question: Explain backpropagation
Enter quiz score (0-100): 40
Enter number of attempts: 3
Enter time spent (minutes): 20

System Output:
{'query_analysis': {'intent': 'Explanation', 'topic': 'Neural Networks', 'difficulty': 'Beginner'}, 'recommendation': {'next_topic': 'Previous fundamentals', 'action': 'Revision', 'difficulty_adjustment': 'Decrease'}}
